In [44]:
os.chdir('C:\\Users\\Vishaal\\Documents\\GitHub\\Hybrid-Transfer-Learning\\Data')

In [45]:
'''

Generic Module Imports 

'''
import time
import os
import copy
import matplotlib.pyplot as plt

In [46]:
'''
    Pennylane is an open source quantum machine learning package by Xanadu. Pennylane also allows to use IBMs Qisklit as an
    installed plugin. We may or may not evntually use that.
    
    Citation: 
    Ville Bergholm, Josh Izaac, Maria Schuld, Christian Gogolin, Carsten Blank, Keri McKiernan and Nathan Killoran. 
    PennyLane. arXiv, 2018. arXiv:1811.04968
'''
import pennylane as qml
from pennylane import numpy as np # Same as standard numpy, but also does automatic differentiation

In [47]:
'''
    PyTorch is an open-source machine learning library by Facebook. It is similar to Tensorflow by Google. It's pretty 
    cool to use this to build models and import pre-trained neural nets like ResNet18. 
    
    PyTorch uses torch tensors and not numpy arrays. We may have to convert as and when necesary. 

'''

import torch
import torch.nn as nn                    #This serves as a base class for neural networks (NNs)#
import torch.optim as optim              #Contains built-in optimizer functions - like Stochastic Gradient Descent(SGD) & Adam Optimizer'''
from torch.optim import lr_scheduler     #Helps modulate learning rate based on number of epochs
import torchvision                       #PyTorchs imgage datasets and other related stuff
from torchvision import datasets, models, transforms

In [48]:
'''

    Initialize Some Parameters - For now we will use an integrated Pennylane Device. This is sort of a simulator for 
    a quantum computer. I will later attempt to use a real Quantum Computer using IBMs Q Experience but no guarantees. 
    This should not technically affect our results. Because we only use 4 qubits, the quantum simulator should have no 
    issues accurately simulating the process in polyomial time. Problems will arise for ~>50 qubits.
    
'''
n_qubits = 4                # Number of qubits
step = 0.0004               # Learning rate
batch_size = 4              # Number of samples for each training step. This is the number of features taken in from ResNet18.
num_epochs = 1              # Number of training epochs. Set to 1 to train quickly. We will later change this to 30
q_depth = 6                 # Depth of the quantum circuit (number of variational layers)
gamma_lr_scheduler = 0.1    # Learning rate reduction applied every 10 epochs.
q_delta = 0.01              # Initial spread of random quantum weights
rng_seed = 0                # Seed for random number generator
start_time = time.time()    # Start of the computation timer


In [49]:
'''
    Initialize the quantum device. Two options for names are Default & Gaussian. Shots is the number of times the circuit 
    will be run. 1024 or 2^10 is generally a good number. Wires is the number of modes to intialise the qubits in. 
    Remember qubits have a probabilistic nature and can take up various states. 
'''
dev = qml.device(name = 'default.qubit', shots = 1024, wires = 4)

In [50]:
'''
    Cuda is a parallel computing architecture created by Nvidia. The following line of code is to check if you have a GPU 
    in your computer and use it. Else, the CPU will be used.
'''
device = torch.device("cuda:0" if torch.cuda.is_available else 'cpu')

In [63]:
'''
    Directory for image data. Note the dataset is relatively small with only 250 images. We cannot train a whole classical
    or quantum CNN using such small data. However, because we are using transfer learning this would suffice to do some
    additional training. 
'''
data_dir = "..\\Data\\hymenoptera_data\\hymenoptera_data"

'''
    Create a dictionary of transforms we need to do to the image data. For both the training and validation set.
    Note: the values for normalise are the mean and std of the images in ImageNet (256 X 256 X 3).  
'''
data_transforms = {
    "train": transforms.Compose(
        [
            transforms.Resize(256),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
        ]
    ),
    "val": transforms.Compose(
        [
            transforms.Resize(256),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
        ]
    ),
}

In [64]:
'''
    Establish the training and validation image datasets. This is the torchvision method of loading data.
'''
image_datasets = {
    x if x == "train" else "validation": datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x])
    for x in ["train", "val"]
}

In [65]:
'''
    244 training images and 153 validation images in two different classes ants and bees
'''
dataset_sizes = {x: len(image_datasets[x]) for x in ["train", "validation"]}
class_names = image_datasets["train"].classes

In [66]:
'''
    Initialize the data loader. We don't load the data yet.
'''
dataloaders = {
    x: torch.utils.data.DataLoader(image_datasets[x], batch_size=batch_size, shuffle=True)
    for x in ["train", "validation"]
}

In [67]:
'''
    Define a function to plot the images as they get ready
'''
def imshow(inp, title=None):
    inp = inp.numpy().transpose((1, 2, 0))
    '''
        Invert the normalisation we did before
    '''
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp)
    if title is not None:
        plt.title(title)

In [69]:
'''
    Designing the Variational Quantum Circuit
'''
'''
    The Hadamard gate applies to a single qubit in state 0 or state 1. Once applied it results in superposition and
    gives a 50% for state 0 or state 1 to exist. Basically - gives qubits their quantum nature.
    https://en.wikipedia.org/wiki/Quantum_logic_gate#Hadamard_(H)_gate
'''

def H_layer(nqubits):
    for idx in range(nqubits):
        qml.Hadamard(wires=idx)
        
'''
    Rotates a single qubit by theta radians along the y-axis. w is a list of angles in radians you want to rotate each
    qubit by
    https://www.quantum-inspire.com/kbase/ry-gate/
    https://pennylane.readthedocs.io/en/stable/code/api/pennylane.RY.html
'''        
def RY_layer(w):
    for idx, element in enumerate(w):
        qml.RY(element, wires=idx)
        
'''
    This is the controlled-not operator. The CNOT gate operates on two qubits at a time. For example, if the first qubit is
    in state 1, the second one is flipped two. In other words, the state of the second qubit depends on the state of the 
    first one. This is kinda what quantum entanglement is. Einstein called it spooky phenomenon and although we have 
    experimental proof, we yet don't have conclusive mathematical proof from first principles to prove this. 
'''
def entangling_layer(nqubits):
    for i in range(0, nqubits - 1, 2):  
        qml.CNOT(wires=[i, i + 1])
    for i in range(1, nqubits - 1, 2):  
        qml.CNOT(wires=[i, i + 1])